# Implementing the `MiniBach` model

## Part 4: Generating an accompaniment for an arbitrary melody

In this step, we use the model trained on Part 3 to generate the three lower voices for a given soprano melodic line.

The model trained during Part 3 has been saved as `trained_model.h5`.  

We can use this model to predict the accompaniment of an arbitrary melody.

In [16]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import music21

In [17]:
model = tf.keras.models.load_model('trained_model.h5')

This is a very rudimentary syntax for encoding our melody:
- Each token represents a sixteenth note
- The special token `--` denotes a *hold* symbol (in the generated scores, it becomes a tie)
- The pipe symbol `|` is just there for visual aid, separating blocks of four sixteenth notes (or one quarter note)
- The same can be said about the newlines, they separate the melody into four blocks (measures) with four quarter notes each

In [57]:
given_melody = '''
A4   --   --   --   |G#4  --   --   --   |A4   --   --   --   |F4   --   --   --   |
D4   --   --   --   |--   --   --   --   |D4   --   --   --   |--   --   --   --   |
E4   --   --   --   |F#4  --   --   --   |G#4  --   --   --   |A4   --   B4   --   |
C5   --   --   --   |C4   --   --   --   |E4   --   --   --   |--   --   --   --   |
'''

In [58]:
given_melody = given_melody.replace('\n', '').replace('|', '')

In [46]:
tokens = given_melody.split()

After organizing the tokens of the input melody, we need to encode it in a one-hot-encoded representation. 

This process is fairly similar to how it was done in Part 2, so I won't describe it here.

In [47]:
SOPRANO_MIN = 57
SOPRANO_MAX = 81

ALTO_MIN = 52
ALTO_MAX = 74

TENOR_MIN = 48
TENOR_MAX = 69

BASS_MIN = 36
BASS_MAX = 64

ranges = {
    'soprano': {midinumber: (midinumber - SOPRANO_MIN + 1) for midinumber in range(SOPRANO_MIN, SOPRANO_MAX + 1)},
    'alto': {midinumber: (midinumber - ALTO_MIN + 1) for midinumber in range(ALTO_MIN, ALTO_MAX + 1)},
    'tenor': {midinumber: (midinumber - TENOR_MIN + 1) for midinumber in range(TENOR_MIN, TENOR_MAX + 1)},
    'bass': {midinumber: (midinumber - BASS_MIN + 1) for midinumber in range(BASS_MIN, BASS_MAX + 1)},
}

reverse_ranges = {
    'soprano': {(midinumber - SOPRANO_MIN + 1): midinumber for midinumber in range(SOPRANO_MIN, SOPRANO_MAX + 1)},
    'alto': {(midinumber - ALTO_MIN + 1): midinumber for midinumber in range(ALTO_MIN, ALTO_MAX + 1)},
    'tenor': {(midinumber - TENOR_MIN + 1): midinumber for midinumber in range(TENOR_MIN, TENOR_MAX + 1)},
    'bass': {(midinumber - BASS_MIN + 1): midinumber for midinumber in range(BASS_MIN, BASS_MAX + 1)},
}

In [48]:
def encode_note(n, rang):
    if n == '--' or n == 'Rest':
        ret = 0
    else:
        note = music21.note.Note(n)
        ret = ranges[rang][note.pitch.midi]
    return ret

def one_hot_encode(idx, rang):
    length = len(ranges[rang].values())
    ret = [0] * (length + 1)
    ret[idx] = 1
    return ret

In [49]:
s = [encode_note(n, 'soprano') for n in tokens] 
x = np.array([[one_hot_encode(idx, 'soprano') for idx in s]])    
x = x.reshape(1, -1)

The melody has been encoded, so we can pass it to the model and collect the predictions from the `MiniBach` model.

In [50]:
predictions = model.predict(x)

predictions = predictions.reshape(-1)

soprano = x.reshape(64, -1)
alto = predictions[:1536].reshape(64, -1)
tenor = predictions[1536:3008].reshape(64, -1)
bass = predictions[3008:4928].reshape(64, -1)

music = {
    'soprano': soprano,
    'alto': alto,
    'tenor': tenor,
    'bass': bass
}

In [51]:
def decode_note(n, rang):
    if n == 0:
        ret = '--'
    else:
        note = music21.note.Note(type='16th')        
        note.pitch.midi = reverse_ranges[rang][n]        
        ret = note
    return ret

In [52]:
generation = {
    'soprano': [],
    'alto': [],
    'tenor': [],
    'bass': []
}

for sixteenth in range(64):
    for part, notes in music.items():
        this_note = decode_note(np.argmax(notes[sixteenth]), part)
        if this_note == '--':
            last_note = generation[part][-1]
            this_note = music21.note.Note(last_note.pitch.nameWithOctave, type='16th')
            if last_note.tie:
                this_note.tie = music21.tie.Tie('continue')
            else:
                last_note.tie = music21.tie.Tie('start')
                generation[part][-1] = last_note
                this_note.tie = music21.tie.Tie('continue')
        else:
            if sixteenth > 0:
                last_note = generation[part][-1]
                if last_note.tie:
                    last_note.tie = music21.tie.Tie('stop')
        generation[part].append(this_note)

The predictions have been generated, decoded, and turned into music notes with the `music21` library. We can give a glance to the 4-part choral (1 given soprano + 3 generated voices).

In [53]:
df = pd.DataFrame(generation)

In [54]:
df

,soprano,alto,tenor,bass
0,<music21.note.Note A>,<music21.note.Note E>,<music21.note.Note A>,<music21.note.Note D>
1,<music21.note.Note A>,<music21.note.Note E>,<music21.note.Note A>,<music21.note.Note D>
2,<music21.note.Note A>,<music21.note.Note E>,<music21.note.Note A>,<music21.note.Note D>
3,<music21.note.Note A>,<music21.note.Note E>,<music21.note.Note A>,<music21.note.Note D>
4,<music21.note.Note G#>,<music21.note.Note E>,<music21.note.Note C>,<music21.note.Note D>
...,...,...,...,...
59,<music21.note.Note A>,<music21.note.Note C#>,<music21.note.Note A>,<music21.note.Note A>
60,<music21.note.Note A>,<music21.note.Note C#>,<music21.note.Note A>,<music21.note.Note A>
61,<music21.note.Note A>,<music21.note.Note C#>,<music21.note.Note A>,<music21.note.Note A>
62,<music21.note.Note A>,<music21.note.Note C#>,<music21.note.Note A>,<music21.note.Note A>


In order to play the score, I use `music21` to generate an output `MusicXML` file.

In [55]:
s = music21.stream.Stream()
s.append(df.soprano.to_list())
a = music21.stream.Stream()
a.append(df.alto.to_list())
t = music21.stream.Stream()
t.append(df.tenor.to_list())
b = music21.stream.Stream()
b.append(df.bass.to_list())
stream = music21.stream.Stream([s,a,t,b])

In [56]:
stream.write('musicxml', 'generated_choral.musicxml')

'/home/napulen/dev/MiniBach/generated_choral.musicxml'

And that's it, a generated choral using the `MiniBach` model.

The `MusicXML` file can be played using a music notation software like MuseScore, Sibelius, Finale, or Dorico.

An alternative option is to export it as `midi`, although midi-generated scores are oftentimes weird looking!

Thanks for checking, and please refer to the original book that describes the `MiniBach` architecture for more details:

> Briot, Jean-Pierre, Gaëtan Hadjeres, and François Pachet. 2017. “Deep Learning Techniques for Music Generation - A Survey.” CoRR abs/1709.01620. http://arxiv.org/abs/1709.01620.

